# Integration tests of square well (refactored)

Here we compare various ways to integrate in Python, both numerically and analytically, using the square well bound-state eigenvector continuation problem as an example.

Something to look at: [quadpy](https://github.com/nschloe/quadpy)

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.integrate import simps, trapz, quad, fixed_quad, quadrature
import matplotlib.pyplot as plt

from scipy.linalg import eig, eigh

from integration import Gaussian_quadrature, Gaussian_mesh
from square_well import SquareWell, rMeshSW, rGaussSW

## Normalization integrals

Here we integrate from 0 to R and from R to r_max (which could be infinity) of the wave function squared.

In [5]:
# specify a test square well
V0 = 2.
R = 1.
mu = 1.
hbar = 1.
new_variable = 5

r_max = 20.

sw_1 = SquareWell(V0, R, mu, hbar, r_max)

In [6]:
new_variable

5

## Scipy routines

### scipy.integrate.quad

An unspecified general integration routine adapted from QUADPACK.
Will work with infinite limits.

In [7]:
# scipy.integrate.quad 
%timeit int_wf_sq_in_quad, err_in = quad(lambda r: sw_1.un_wf_in(r)**2, \
                                 0, R, epsabs=1.e-14, epsrel=1.e-14)
%timeit int_wf_sq_out_quad, err_out = quad(lambda r: sw_1.un_wf_out(r)**2, \
                                   R, r_max, epsabs=1.e-14, epsrel=1.e-14)
print(f'inner by quad {int_wf_sq_in_quad:.16f} +/- {err_in:.3e}')
print(f'outer by quad {int_wf_sq_out_quad:.16f} +/- {err_out:.3e}')

49.5 µs ± 414 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
229 µs ± 743 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


NameError: name 'int_wf_sq_in_quad' is not defined

In [3]:
# scipy.integrate.quad 
int_wf_sq_in_quad, err_in = quad(lambda r: sw_1.un_wf_in(r)**2, 
                                 0, R, epsabs=1.e-14, epsrel=1.e-14)
int_wf_sq_out_quad, err_out = quad(lambda r: sw_1.un_wf_out(r)**2, 
                                   R, r_max, epsabs=1.e-14, epsrel=1.e-14)
print(f'inner by quad {int_wf_sq_in_quad:.16f} +/- {err_in:.3e}')
print(f'outer by quad {int_wf_sq_out_quad:.16f} +/- {err_out:.3e}')

inner by quad 0.5797556310356548 +/- 6.437e-15
outer by quad 0.2187357524270836 +/- 2.432e-15


Now try r_max -> np.inf. It works!

In [4]:
# Now take it to r_max -> infinity
# scipy.integrate.quad 
int_wf_sq_in_quad, err_in = quad(lambda r: sw_1.un_wf_in(r)**2, 
                                 0, R, epsabs=1.e-14, epsrel=1.e-14)
int_wf_sq_out_quad, err_out = quad(lambda r: sw_1.un_wf_out(r)**2, 
                                   R, np.inf, epsabs=1.e-14, epsrel=1.e-14)
print(f'inner by quad {int_wf_sq_in_quad:.16f} +/- {err_in:.3e}')
print(f'outer by quad {int_wf_sq_out_quad:.16f} +/- {err_out:.3e}')

inner by quad 0.5797556310356548 +/- 6.437e-15
outer by quad 0.2187357524335422 +/- 4.739e-15


### scipy.integrate.quadrature

Computes a definite integral using fixed-tolerance Gaussian quadrature. 
Only works with finite limits. 

In [5]:
# scipy.integrate.quadrature
int_wf_sq_in_quadrature, err = quadrature(lambda r: sw_1.un_wf_in(r)**2, 
                                          0, R, tol=1.e-14, rtol=1.e-14)
int_wf_sq_out_quadrature, err = quadrature(lambda r: sw_1.un_wf_out(r)**2, 
                                          R, r_max, tol=1.e-14, rtol=1.e-14)
print(f'inner by quadrature {int_wf_sq_in_quadrature:.16f} +/- {err:.3e}')
print(f'outer by quadrature {int_wf_sq_out_quadrature:.16f} +/- {err:.3e}')

inner by quadrature 0.5797556310356546 +/- 1.527e-15
outer by quadrature 0.2187357524270815 +/- 1.527e-15


### scipy.integrate.fixed_quad

Computes a definite integral using fixed-order Gaussian quadrature.
Only works with finite limits. 

In [6]:
# scipy.integrate.fixed_quad
int_wf_sq_in_fixed_quad, dum = fixed_quad(lambda r: sw_1.un_wf_in(r)**2, 
                                     0, R, n=20)
int_wf_sq_out_fixed_quad, dum = fixed_quad(lambda r: sw_1.un_wf_out(r)**2, 
                                     0, R, n=20)
print(f'inner by fixed_quad {int_wf_sq_in_fixed_quad:.16f} ')
print(f'outer by fixed_quad {int_wf_sq_out_fixed_quad:.16f} ')

inner by fixed_quad 0.5797556310356546 
outer by fixed_quad 0.5649079751430025 


### Uniform mesh 

Use Simpson's rule (`scipy.integrate.simps`) or the trapezoid rule (`scipy.integrate.trapz`).

In [7]:
# Use a mesh with equal points from 0 to R and R to r_max
num_pts = 1000
r_mesh = rMeshSW(R, r_max, num_pts, num_pts)

wf_sq_in = sw_1.un_wf_in(r_mesh.r_in_pts)**2
int_wf_sq_in_simps = simps(wf_sq_in, r_mesh.r_in_pts)
int_wf_sq_in_trapz = trapz(wf_sq_in, r_mesh.r_in_pts)

wf_sq_out = sw_1.un_wf_out(r_mesh.r_out_pts)**2
int_wf_sq_out_simps = simps(wf_sq_out, r_mesh.r_out_pts)
int_wf_sq_out_trapz = trapz(wf_sq_out, r_mesh.r_out_pts)

print(rf'Uniform mesh up to r_max = {r_max:.1f} with {num_pts} points.')
print(rf'inner integration Simpsons rule:  {int_wf_sq_in_simps:.16f}')
print(rf'inner integration trapezoid rule: {int_wf_sq_in_trapz:.16f}')
print(rf'outer integration Simpsons rule:  {int_wf_sq_out_simps:.16f}')
print(rf'outer integration trapezoid rule: {int_wf_sq_out_trapz:.16f}')

Uniform mesh up to r_max = 20.0 with 1000 points.
inner integration Simpsons rule:  0.5797556310965298
inner integration trapezoid rule: 0.5797555353263462
outer integration Simpsons rule:  0.2187358815657248
outer integration trapezoid rule: 0.2187464891851396


In [8]:
# Make a table comparing Simpson's and trapezoid to one of the best routines.

best_in = int_wf_sq_in_quadrature
best_out = int_wf_sq_out_quadrature

print('Inner wave function:')
print(' # pts      Simpsons       relerr',
      '          Trapezoid        relerr')
for n_pts in [100, 200, 400, 800, 1600]:
    r_mesh = rMeshSW(R, r_max, n_pts, n_pts)
    wf_sq_in = sw_1.un_wf_in(r_mesh.r_in_pts)**2
    simpsons = simps(wf_sq_in, r_mesh.r_in_pts)
    rel_simps = np.abs((simpsons - best_in) / best_in)
    trapezoid = trapz(wf_sq_in, r_mesh.r_in_pts)
    rel_trapz = np.abs((trapezoid - best_in) / best_in)
    print(f' {n_pts:4d}  {simpsons:.16f} {rel_simps:.2e}',
          f'    {trapezoid:.16f}   {rel_trapz:.2e}')

print('\n')
print('Outer wave function:')
print(' # pts      Simpsons       relerr',
      '          Trapezoid        relerr')
for n_pts in [100, 200, 400, 800, 1600]:
    r_mesh = rMeshSW(R, r_max, n_pts, n_pts)
    wf_sq_out = sw_1.un_wf_out(r_mesh.r_out_pts)**2
    simpsons = simps(wf_sq_out, r_mesh.r_out_pts)
    rel_simps = np.abs((simpsons - best_out) / best_out)
    trapezoid = trapz(wf_sq_out, r_mesh.r_out_pts)
    rel_trapz = np.abs((trapezoid - best_out) / best_out)
    print(f' {n_pts:4d}  {simpsons:.16f} {rel_simps:.2e}',
          f'    {trapezoid:.16f}   {rel_trapz:.2e}')



Inner wave function:
 # pts      Simpsons       relerr           Trapezoid        relerr
  100  0.5797556912185021 1.04e-07     0.5797458850610347   1.68e-05
  200  0.5797556386084631 1.31e-08     0.5797532190122070   4.16e-06
  400  0.5797556319851576 1.64e-09     0.5797550310513172   1.03e-06
  800  0.5797556311545172 2.05e-10     0.5797554814149639   2.58e-07
 1600  0.5797556310505231 2.56e-11     0.5797555936772636   6.44e-08


Outer wave function:
 # pts      Simpsons       relerr           Trapezoid        relerr
  100  0.2188582471476681 5.60e-04     0.2198279577212179   4.99e-03
  200  0.2187515196434340 7.21e-05     0.2190062695943003   1.24e-03
  400  0.2187377524908481 9.14e-06     0.2188030556543204   3.08e-04
  800  0.2187360042801514 1.15e-06     0.2187525369183738   7.67e-05
 1600  0.2187357840247480 1.44e-07     0.2187399433513712   1.92e-05


## Gaussian quadrature

In [9]:
best_in = int_wf_sq_in_quadrature
best_out = int_wf_sq_out_quadrature

a = 0.
b = R
type = 'Legendre'

print('Inner wave function:')
print(' # pts      Gauss-Legendre    relerr    ')
for n_pts in [12, 24, 36, 48, 64]:
    nodes, weights = Gaussian_quadrature(n_pts, type=type, 
                                         a=a, b=b)
    Legendre = sw_1.un_wf_in(nodes)**2 @ weights
    rel_Legendre = np.abs((Legendre - best_in)/best_in)
    print(f' {n_pts:4d}     {Legendre:.16f}',
          f' {rel_Legendre:.3e}  ')


a = R
b = r_max
type = 'Legendre'

print('\n')
print('Outer wave function:')
print(' # pts      Gauss-Legendre    relerr    ')
for n_pts in [12, 24, 36, 48, 64]:
    nodes, weights = Gaussian_quadrature(n_pts, type=type, 
                                         a=a, b=b)
    Legendre = sw_1.un_wf_out(nodes)**2 @ weights
    rel_Legendre = np.abs((Legendre - best_out)/best_out)
    print(f' {n_pts:4d}     {Legendre:.16f}',
          f' {rel_Legendre:.3e}  ')
    

Inner wave function:
 # pts      Gauss-Legendre    relerr    
   12     0.5797556310356546  0.000e+00  
   24     0.5797556310356549  3.830e-16  
   36     0.5797556310356544  3.830e-16  
   48     0.5797556310356542  7.660e-16  
   64     0.5797556310356548  1.915e-16  


Outer wave function:
 # pts      Gauss-Legendre    relerr    
   12     0.2187357507135612  7.834e-09  
   24     0.2187357524270832  7.360e-15  
   36     0.2187357524270865  2.271e-14  
   48     0.2187357524270915  4.568e-14  
   64     0.2187357524270831  7.233e-15  


## Sympy

This is a work in progress . . .

In [10]:
import sympy as sym
from sympy.utilities.lambdify import lambdify, implemented_function

r_s, k_in_s = sym.symbols('r_s k_in_s')
expr = sym.integrate(sym.sin(k_in_s*r_s)**2, r_s)

f = lambdify([r_s, k_in_s], expr, "numpy")

In [11]:
print(expr)

Piecewise(((k_in_s*r_s/2 - sin(k_in_s*r_s)*cos(k_in_s*r_s)/2)/k_in_s, Ne(k_in_s, 0)), (0, True))


## Basic tests of Gaussian quadrature

Some simple tests that the nodes and weights are correct.

In [12]:
npts = 30
lower_limit = 0.
upper_limit = 2.
type = 'Legendre'
nodes, weights = Gaussian_quadrature(npts, type=type, 
                                     a=lower_limit, b=upper_limit)

print(nodes)
print(weights)

[0.00310652 0.01633188 0.03997814 0.07379995 0.11743946 0.17043424
 0.23222257 0.30214951 0.37947382 0.46337585 0.55296623 0.64729527
 0.74536307 0.84613009 0.94852816 1.05147184 1.15386991 1.25463693
 1.35270473 1.44703377 1.53662415 1.62052618 1.69785049 1.76777743
 1.82956576 1.88256054 1.92620005 1.96002186 1.98366812 1.99689348]
[0.00796819 0.01846647 0.02878471 0.03879919 0.04840267 0.05749316
 0.06597423 0.07375597 0.0807559  0.08689979 0.09212252 0.09636874
 0.09959342 0.10176239 0.10285265 0.10285265 0.10176239 0.09959342
 0.09636874 0.09212252 0.08689979 0.0807559  0.07375597 0.06597423
 0.05749316 0.04840267 0.03879919 0.02878471 0.01846647 0.00796819]


In [13]:
# integral of x from 0 to 2 --> 2
integrand = nodes
exact = 2
answer = integrand @ weights
print(f'Integral of x from 0 to 2 [{exact}]: {answer}', 
      f' {np.abs((exact-answer)/exact):.3e}')

# integral of x^2 from 0 to 2 --> 8/3
integrand = nodes**2
exact = 8/3
answer = integrand @ weights
print(f'Integral of x^2 from 0 to 2 [{exact}]: {answer}', 
      f' {np.abs((exact-answer)/exact):.3e}')

# integral of e^{-x} from zero to 2 
exact = 1 - np.exp(-2);
integrand = np.exp(-nodes)
answer = integrand @ weights
print(f'Integral of exp(-x) from 0 to 2 [{exact}]: {answer}', 
      f' {np.abs((exact-answer)/exact):.3e}')

# integral of sin(10x) from zero to 2 
exact = (1 - np.cos(20))/10;
integrand = np.sin(10*nodes)
answer = integrand @ weights
print(f'Integral of sin(10x) from 0 to 2 [{exact}]: {answer}', 
      f' {np.abs((exact-answer)/exact):.3e}')



Integral of x from 0 to 2 [2]: 1.9999999999999998  1.110e-16
Integral of x^2 from 0 to 2 [2.6666666666666665]: 2.66666666666667  1.332e-15
Integral of exp(-x) from 0 to 2 [0.8646647167633873]: 0.864664716763388  7.704e-16
Integral of sin(10x) from 0 to 2 [0.0591917938186608]: 0.059191793818664046  5.486e-14


In [14]:
# Let's do some basic checks
npts_array = [15, 10, 12]
lower_limit_array = [0., 1., 1.5]
upper_limit_array = [1., 1.5, 2.]
type = 'Legendre'
nodes, weights = Gaussian_mesh(npts_array, type, lower_limit_array,
                               upper_limit_array);

# integral of x from 0 to 2 --> 2
integrand = nodes
exact = 2
answer = integrand @ weights
print(f'Integral of x from 0 to 2 [{exact}]: {answer}', 
      f' {np.abs((exact-answer)/exact):.3e}')

# integral of x^2 from 0 to 2 --> 8/3
integrand = nodes**2
exact = 8/3
answer = integrand @ weights
print(f'Integral of x^2 from 0 to 2 [{exact}]: {answer}', 
      f' {np.abs((exact-answer)/exact):.3e}')

# integral of e^{-x} from zero to 2 
exact = 1 - np.exp(-2);
integrand = np.exp(-nodes)
answer = integrand @ weights
print(f'Integral of exp(-x) from 0 to 2 [{exact}]: {answer}', 
      f' {np.abs((exact-answer)/exact):.3e}')

# integral of sin(10x) from zero to 2 
exact = (1 - np.cos(20))/10;
integrand = np.sin(10*nodes)
answer = integrand @ weights
print(f'Integral of sin(10x) from 0 to 2 [{exact}]: {answer}', 
      f' {np.abs((exact-answer)/exact):.3e}')



Integral of x from 0 to 2 [2]: 2.0  0.000e+00
Integral of x^2 from 0 to 2 [2.6666666666666665]: 2.666666666666667  1.665e-16
Integral of exp(-x) from 0 to 2 [0.8646647167633873]: 0.8646647167633872  1.284e-16
Integral of sin(10x) from 0 to 2 [0.0591917938186608]: 0.05919179381866042  6.448e-15
